In [1]:
from google.cloud import bigquery
from datetime import timedelta
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import re
import pickle
import json
import datetime as dt
from pins import board_rsconnect
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from collections import Counter


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 


def GBQ_data(query_string):
    client = bigquery.Client('turing-230020')
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()


In [2]:
API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
job_data = board.pin_read("muhammad_usman/demand-basic-data")
print(job_data.shape)
job_data.head()

(9593, 52)


,job_id,company,role,role_type,is_deleted,max_acceptable_rate,job_value,number_of_open_roles,opportunity_status,status,must_have_total_years_of_experience,nice_have_total_years_of_experience,priority,opportunityid,opportunity_created_date,client_type,leadsource,client_category,region,chosen_type,slp_date,pcp_date,psp_date,ir_date,is_date,ih_date,dc_date,signed_date,trial_date,start_date,chosen_dev_id,dc14,total_positions,total_open_positions__c,total_positions_by_account,must_have_skill_ids,must_have_skill_names,must_have_skill_ids_tuple,num_must_have_skills,nice_have_skill_ids,nice_have_skill_names,nice_have_skill_ids_tuple,num_nice_have_skills,ms_num_queries,ss_queries,queries,interviews_requested,interviews_scheduled,interviews_happened,interviews_passed,interviews_failed,engagement_days
0,59,ELSA,Full-Stack Python / Vue OR React,NaN,1,NaN,NaN,1.0,NaN,Creating Shortlist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55,Cloud One,Senior Full-Stack PHP + JS,NaN,0,NaN,3.9,1.0,NaN,Trial Succeeded,NaN,NaN,NaN,0061U00000DeXrCQAV,2019-11-28 18:58:20+00:00,existing,existing,4.Bronze,2.Rest of US,matching,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-10 22:38:10+00:00,2019-12-12 00:00:00+00:00,2019-12-12 00:00:00+00:00,136695.0,1.0,0.0,NaN,3.0,[ 93 165],['JavaScript' 'PHP'],"(93, 165)",2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.0
2,3,treehubapp,Full stack,NaN,0,NaN,7.4,1.0,NaN,Trial Succeeded,NaN,NaN,NaN,0061U00000E8sz5QAB,2019-12-11 02:37:57+00:00,new,new,4.Bronze,2.Rest of US,matching,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,2019-12-19 18:10:21+00:00,2019-12-18 00:00:00+00:00,2019-12-18 00:00:00+00:00,249794.0,1.0,1.0,NaN,1.0,[ 120 2031],['Node.js' 'React'],"(120, 2031)",2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,469.0
3,7,Jisr.net,Full-Stack Rails Developer,NaN,1,NaN,NaN,1.0,NaN,Creating Shortlist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,112,Chad Connects,Front-End Vue.js Developer,NaN,1,NaN,NaN,1.0,NaN,Preparing Packet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
job_data_cols = ['job_id', 'company', 'client_category','opportunity_created_date', 'dc_date', 'max_acceptable_rate', 'is_deleted',
                 'job_value', 'status', 'num_must_have_skills', 'must_have_total_years_of_experience',
                'ms_num_queries', 'ss_queries', 'queries', 'interviews_requested', 'interviews_scheduled',
                'interviews_happened','interviews_passed', 'interviews_failed', 'region', 'engagement_days',
                'number_of_open_roles', 'total_positions', 'total_open_positions__c', 'must_have_skill_ids', 'must_have_skill_names']

final_data = job_data[job_data_cols].copy()
#final_data['opportunity_created_date'] = pd.to_datetime(final_data['opportunity_created_date'])
final_data['dc_date'] = pd.to_datetime(final_data['dc_date'])
final_data['dc'] = np.where(final_data['dc_date'].isna(), 0,1)
#final_data['ninety_days_old'] = np.where((final_data['opportunity_created_date'] > dt.datetime.now(pytz.utc) - timedelta(days = 90)) & (final_data['dc']==0) , 1,0)
#final_data['time_taken_DC'] = (final_data['dc_date'] - final_data['opportunity_created_date']).dt.days
#final_data['time_taken_DC'] = np.where(final_data['time_taken_DC']<0, 0, final_data['time_taken_DC'])
final_data['enterprise'] = np.where(final_data['client_category']=='1.Platinum', 1,0)
final_data['active'] = np.where(~(final_data['status'].isin(['Lost Opportunity', 'Project Aborted', 'Paused'])) & (final_data['dc']==0), 1,0)
final_data['active_roles'] = np.where(final_data['active']==1, final_data['total_positions'],0)
final_data['lost'] = np.where((final_data['status'].isin(['Lost Opportunity', 'Project Aborted', 'Paused']) & (final_data['dc']==0)), 1,0)
final_data['max_acceptable_rate'] = np.where(final_data['max_acceptable_rate'] >300 , 300, final_data['max_acceptable_rate'])
final_data['total_positions'] = np.where(final_data['total_positions']==0 , 1,final_data['total_positions'])
final_data['is_bulk'] = np.where(final_data['total_positions']>2 , 1,0)
#final_data = final_data.merge(pckt_sent, how='left', on='job_id')
#final_data['is_pckt_sent'] = np.where(final_data['num_pckt_sent'].isna(), 0,1)

In [8]:
final_data

,job_id,company,client_category,opportunity_created_date,dc_date,max_acceptable_rate,is_deleted,job_value,status,num_must_have_skills,must_have_total_years_of_experience,ms_num_queries,ss_queries,queries,interviews_requested,interviews_scheduled,interviews_happened,interviews_passed,interviews_failed,region,engagement_days,number_of_open_roles,total_positions,total_open_positions__c,must_have_skill_ids,must_have_skill_names,dc,enterprise,active,active_roles,lost,is_bulk
0,59,ELSA,NaN,NaN,NaT,NaN,1,NaN,Creating Shortlist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0
1,55,Cloud One,4.Bronze,2019-11-28 18:58:20+00:00,2019-12-10 22:38:10+00:00,NaN,0,3.9,Trial Succeeded,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.Rest of US,103.0,1.0,1.0,NaN,[ 93 165],['JavaScript' 'PHP'],1,0,0,0.0,0,0
2,3,treehubapp,4.Bronze,2019-12-11 02:37:57+00:00,2019-12-11 02:37:57+00:00,NaN,0,7.4,Trial Succeeded,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.Rest of US,469.0,1.0,1.0,NaN,[ 120 2031],['Node.js' 'React'],1,0,0,0.0,0,0
3,7,Jisr.net,NaN,NaN,NaT,NaN,1,NaN,Creating Shortlist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0
4,112,Chad Connects,NaN,NaN,NaT,NaN,1,NaN,Preparing Packet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,0,1,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9588,7569,HigherEdLab.com,NaN,NaN,NaT,NaN,0,NaN,Soliciting Further Requirements,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2036],['Angular'],0,0,1,NaN,0,0
9589,6184,unboring company,NaN,NaN,NaT,NaN,0,NaN,Paused,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,[2031],['React'],0,0,0,0.0,1,0
9590,6188,HigherEdLab.com,NaN,NaN,NaT,NaN,0,NaN,Creating Workflow,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2031],['React'],0,0,1,NaN,0,0
9591,7989,Turing,NaN,NaN,NaT,NaN,0,NaN,Creating Shortlist,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[2036],['Angular'],0,0,1,NaN,0,0


In [9]:
import pandas_gbq
cd_date = GBQ_data("""Select * from turing-dev-337819.pdsa.demand_cluster """)
print(cd_date.shape)
cd_date.head()

(7275, 5)


,job_id,client_category,cluster,developer_chosen,cluster_old
0,70,2.Gold,1,1.0,0
1,823,2.Gold,1,0.0,0
2,954,2.Gold,1,0.0,0
3,1019,2.Gold,1,0.0,0
4,1067,2.Gold,1,1.0,0


In [20]:
duck = cd_date.merge(final_data, how='left', on='job_id')
duck.head()

,job_id,client_category_x,cluster,developer_chosen,cluster_old,company,client_category_y,opportunity_created_date,dc_date,max_acceptable_rate,is_deleted,job_value,status,num_must_have_skills,must_have_total_years_of_experience,ms_num_queries,ss_queries,queries,interviews_requested,interviews_scheduled,interviews_happened,interviews_passed,interviews_failed,region,engagement_days,number_of_open_roles,total_positions,total_open_positions__c,must_have_skill_ids,must_have_skill_names,dc,enterprise,active,active_roles,lost,is_bulk
0,70,2.Gold,1,1.0,0,Proxy,2.Gold,2019-11-03 19:53:49+00:00,2019-11-03 19:53:49+00:00,10.0,0,23.4,Trial Succeeded,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.Rest of US,344.0,1.0,1.0,NaN,[2031],['React'],1,0,0,0.0,0,0
1,823,2.Gold,1,0.0,0,Pangaea Holdings,2.Gold,2020-08-20 00:39:36+00:00,NaT,35.0,0,23.4,Trial Succeeded,1.0,NaN,NaN,NaN,NaN,4.0,2.0,2.0,0.0,1.0,2.Rest of US,698.0,2.0,1.0,NaN,[2031],['React'],0,0,1,1.0,0,0
2,954,2.Gold,1,0.0,0,MotoRefi,2.Gold,2020-09-15 17:36:13+00:00,NaT,25.0,0,13.0,Lost Opportunity,1.0,NaN,NaN,NaN,NaN,2.0,1.0,1.0,0.0,1.0,2.Rest of US,NaN,1.0,1.0,NaN,[2031],['React'],0,0,0,0.0,1,0
3,1019,2.Gold,1,0.0,0,Narvar,2.Gold,2020-10-05 21:09:59+00:00,NaT,35.0,0,22.2,Lost Opportunity,1.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,0.0,2.0,1.Bay Area,NaN,1.0,1.0,NaN,[2031],['React'],0,0,0,0.0,1,0
4,1067,2.Gold,1,1.0,0,RTIC,2.Gold,2020-10-14 13:38:47+00:00,2020-10-21 20:19:14+00:00,35.0,0,23.4,Lost Opportunity,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,2.Rest of US,28.0,1.0,2.0,NaN,[2031],['React'],1,0,0,0.0,0,0


In [25]:
duck = duck.loc[duck['cluster']==2]
duck[['job_id', 'client_category_x', 'cluster', 'company','active', 'must_have_skill_names']].to_csv('data_req_final.csv')